In [2]:
import numpy as np

def viterbi_algorithm(S, Pi, E, P, Y):
    K = len(S)
    T = len(Y)
    
    # Initialize matrices for Viterbi probabilities and paths
    viterbi_prob = np.zeros((K, T))
    viterbi_path = np.zeros((K, T))
    
    # Part I: Initialization
    for i in range(K):
        viterbi_prob[i, 0] = Pi[i] * E[i, Y[0]]
    
    # Part II: Compute Viterbi probabilities and paths
    for j in range(1, T):
        for i in range(K):
            viterbi_prob[i, j] = max(E[i, Y[j]] * P[k, i] * viterbi_prob[k, j-1] for k in range(K))
            viterbi_path[i, j] = np.argmax([E[i, Y[j]] * P[k, i] * viterbi_prob[k, j-1] for k in range(K)])
    
    # Part III: Re-track the most likely path
    X = []
    xT = np.argmax(viterbi_prob[:, T-1])
    X.append(xT)
    for j in range(T-1, 0, -1):
        x_prev = int(viterbi_path[int(xT), j])
        X.append(x_prev)
        xT = x_prev
    
    # Reverse X to get the correct order
    X.reverse()
    
    return X, np.max(viterbi_prob[:, T-1])

def forward_algorithm(S, Pi, E, P, Y):
    K = len(S)
    T = len(Y)
    
    # Initialize matrix for forward probabilities
    forward_prob = np.zeros((K, T))
    
    # Part I: Initialization
    for i in range(K):
        forward_prob[i, 0] = Pi[i] * E[i, Y[0]]
    
    # Part II: Recursion
    for t in range(1, T):
        for i in range(K):
            forward_prob[i, t] = sum(forward_prob[k, t-1] * P[k, i] * E[i, Y[t]] for k in range(K))
    
    # Part III: Termination
    probability = np.sum(forward_prob[:, T-1])
    
    return probability

# Example usage
S = ['rainy', 'cloudy', 'sunny']
Pi = [0.35, 0.25, 0.4]
P = np.array([[0.4, 0.2, 0.4], [0.2, 0.5, 0.3], [0.3, 0.6, 0.1]])
E = np.array([[0.8, 0.1, 0.1], [0.2, 0.5, 0.3], [0.4, 0.2, 0.4]])
Y = [0, 0, 2, 1, 2]  # Mapping: read=0, shop=1, play=2

# Viterbi algorithm
weather_sequence, max_probability = viterbi_algorithm(S, Pi, E, P, Y)
print("Most likely weather sequence:", [S[i] for i in weather_sequence])
print("Max probability:", max_probability)

# Forward algorithm
probability = forward_algorithm(S, Pi, E, P, Y)
print("Probability of observed sequence:",probability)

Most likely weather sequence: ['rainy', 'rainy', 'sunny', 'cloudy', 'cloudy']
Max probability: 0.0006451200000000002
Probability of observed sequence: 0.0054941706


In [4]:
import numpy as np

def viterbi(obs, states, start_prob, trans_prob, emit_prob):
    """
    Viterbi algorithm to find the most likely sequence of hidden states given observed sequence.
    
    Args:
        obs: List of observed symbols.
        states: List of hidden states.
        start_prob: Initial probability distribution over states.
        trans_prob: Transition probability matrix.
        emit_prob: Emission probability matrix.
        
    Returns:
        Tuple containing:
        - Most likely sequence of hidden states.
        - Probability of the most likely sequence.
    """
    n_obs = len(obs)
    n_states = len(states)
    
    # Initialize viterbi matrix and backpointer matrix
    viterbi_matrix = np.zeros((n_states, n_obs))
    backpointer = np.zeros((n_states, n_obs), dtype=int)
    
    # Initialize first column of viterbi matrix
    viterbi_matrix[:, 0] = start_prob * emit_prob[:, obs[0]]
    
    # Iterate over each observation
    for t in range(1, n_obs):
        # Iterate over each state
        for s in range(n_states):
            # Calculate probability of transitioning to state s from all other states
            trans_probs = viterbi_matrix[:, t-1] * trans_prob[:, s]
            
            # Find the maximum probability and corresponding state
            max_prob_state = np.argmax(trans_probs)
            max_prob = trans_probs[max_prob_state]
            
            # Update viterbi matrix and backpointer
            viterbi_matrix[s, t] = max_prob * emit_prob[s, obs[t]]
            backpointer[s, t] = max_prob_state
    
    # Reconstruct the most likely sequence of states
    best_sequence = [np.argmax(viterbi_matrix[:, -1])]
    for t in range(n_obs - 1, 0, -1):
        best_sequence.append(backpointer[best_sequence[-1], t])
    best_sequence.reverse()
    
    # Calculate probability of the best sequence
    best_prob = np.max(viterbi_matrix[:, -1])
    
    return best_sequence, best_prob

# Example usage:
observed_sequence = [0, 1, 0]  # Observed sequence of symbols
hidden_states = [0, 1]         # Hidden states
start_probability = np.array([0.5, 0.5])  # Initial probability distribution
transition_probability = np.array([[0.7, 0.3], [0.4, 0.6]])  # Transition probabilities
emission_probability = np.array([[0.9, 0.1], [0.2, 0.8]])      # Emission probabilities

# Run the Viterbi algorithm
sequence, probability = viterbi(observed_sequence, hidden_states, start_probability, transition_probability, emission_probability)

print("Most likely sequence of hidden states:",sequence)
print("Probability of the most likely sequence:",probability)


Most likely sequence of hidden states: [0, 1, 0]
Probability of the most likely sequence: 0.03888000000000001


In [6]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"] * trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
    return V

def backtrace(V):
    # Find the state with the highest final probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its index in the last observation
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            previous = st
            break
    # Backtrack to find the path
    path = [previous]
    for t in range(len(V) - 2, -1, -1):
        previous = V[t + 1][previous]["prev"]
        path.insert(0, previous)
    return path

states = ('Rainy', 'Sunny')
observations = ('walk', 'shop', 'clean')
start_probability = {'Rainy': 0.6, 'Sunny': 0.4}
transition_probability = {'Rainy': {'Rainy': 0.7, 'Sunny': 0.3}, 'Sunny': {'Rainy': 0.4, 'Sunny': 0.6}}
emission_probability = {'Rainy': {'walk': 0.1, 'shop': 0.4, 'clean': 0.5}, 'Sunny': {'walk': 0.6, 'shop': 0.3, 'clean': 0.1}}

V = viterbi(observations, states, start_probability, transition_probability, emission_probability)
best_path = backtrace(V)
print("The most likely states are:",best_path)

The most likely states are: ['Sunny', 'Rainy', 'Rainy']
